# STL Export Demo: From Visualization to 3D Print

This notebook demonstrates how to export complex function visualizations as STL files for 3D printing.

We'll:
1. Visualize a complex function on the Riemann sphere
2. Export it as an STL file suitable for 3D printing
3. Explain the key settings and options

In [ ]:
import complexplorer as cp
import numpy as np

# Check if PyVista is available
try:
    import pyvista as pv
    print("✓ PyVista is installed - STL export will work!")
except ImportError:
    print("✗ PyVista not found. Install with: pip install pyvista")

## Step 1: Define and Visualize a Complex Function

Let's start with a beautiful rational function that has interesting features on the Riemann sphere.

In [ ]:
# Define our complex function
def f(z):
    """A rational function with poles and zeros."""
    return (z**3 - 1) / (z**2 + z + 1)

# Create a colormap for visualization
cmap = cp.Phase(n_phi=12, auto_scale_r=True, v_base=0.4)

print("Function: f(z) = (z³ - 1) / (z² + z + 1)")
print("This function has:")
print("- Zeros at the cube roots of unity: 1, e^(2πi/3), e^(4πi/3)")
print("- Poles where z² + z + 1 = 0")

In [ ]:
# Visualize on the Riemann sphere
# Note: In Jupyter, we use trame backend which may have aliasing
plotter = cp.riemann_pv(
    f,
    resolution=150,  # Lower resolution for notebook
    cmap=cmap,
    modulus_mode='arctan',  # Smooth compression of modulus
    notebook=True,  # Jupyter display
    window_size=(600, 600),
    return_plotter=True
)

## Step 2: Export to STL for 3D Printing

Now let's export this beautiful visualization as an STL file that can be 3D printed!

In [ ]:
from complexplorer.export.stl import OrnamentGenerator

# Create the ornament generator
ornament = OrnamentGenerator(
    func=f,
    resolution=200,  # Higher resolution for quality STL
    scaling='arctan',  # Modulus scaling method
    cmap=cmap,
    domain=None  # Use full sphere
)

print("Ornament generator created with:")
print(f"- Resolution: {ornament.resolution} (affects mesh density)")
print(f"- Scaling: {ornament.scaling} (how modulus maps to radius)")
print(f"- Colormap: Enhanced phase portrait with {cmap.n_phi} sectors")

### Understanding the Settings

**Resolution**: Controls the mesh density
- Higher values (200-300) = smoother surface, larger file
- Lower values (100-150) = faster generation, smaller file

**Scaling Methods**:
- `'constant'`: Traditional sphere (radius = 1)
- `'arctan'`: Smooth compression, good for functions with poles
- `'logarithmic'`: Emphasizes zeros and poles
- `'linear_clamp'`: Linear up to a maximum

**Domain**: Restrict which parts of the sphere to include
- `None`: Full sphere
- `cp.Disk(5)`: Only |z| < 5
- `cp.Annulus(0.1, 10)`: Exclude origin (good for poles at z=0)

In [ ]:
# Generate the STL file
stl_file = ornament.generate_and_save(
    filename='complex_ornament.stl',
    size_mm=80,  # Physical size in millimeters
    repair=True,  # Apply mesh repair for better print quality
    verbose=True  # Show progress
)

print(f"\n✓ STL file created: {stl_file}")
print(f"\nYou can now import this file into your 3D printing software!")

## Step 3: Customize for Different Functions

Let's try another function with different characteristics:

In [ ]:
# A function with essential singularity
def g(z):
    """Exponential function - essential singularity at infinity."""
    return np.exp(z)

# For functions with rapid growth, restrict the domain
restricted_domain = cp.Disk(3)  # Only consider |z| < 3

# Create ornament with domain restriction
ornament2 = OrnamentGenerator(
    func=g,
    resolution=150,
    scaling='logarithmic',  # Good for exponential growth
    cmap=cp.Phase(n_phi=8, auto_scale_r=True),
    domain=restricted_domain
)

# Generate STL
stl_file2 = ornament2.generate_and_save(
    filename='exponential_ornament.stl',
    size_mm=70,
    repair=True,
    verbose=True
)

print(f"✓ Created: {stl_file2}")
print("\nDomain restriction helps with:")
print("- Numerical stability")
print("- Avoiding infinite values")
print("- Creating cleaner 3D prints")

## Tips for 3D Printing

1. **File Size**: STL files can be large. Resolution 150-200 is usually sufficient.

2. **Printing Settings**:
   - The base is automatically flattened for easy printing
   - No supports needed for most ornaments
   - Use 0.2mm layer height for good detail

3. **Material Choice**:
   - PLA works well for display pieces
   - PETG for outdoor ornaments
   - Resin printing for maximum detail

4. **Scaling in Slicer**:
   - The `size_mm` parameter sets the diameter
   - You can rescale in your slicer software

5. **Color**: 
   - STL files don't include color information
   - Consider painting or using multi-color printing techniques

## Advanced: Custom Scaling Function

For complete control over the 3D shape, you can define custom scaling:

In [ ]:
# Define a custom scaling function
def custom_scale(moduli):
    """Custom scaling that emphasizes mid-range values."""
    # Sigmoid-like scaling
    return 1 / (1 + np.exp(-2 * (moduli - 1)))

# Create ornament with custom scaling
ornament3 = OrnamentGenerator(
    func=lambda z: z**4 - 1,  # Simple polynomial
    resolution=180,
    scaling='custom',
    scaling_params={'scaling_func': custom_scale},
    cmap=cp.Phase(n_phi=16, auto_scale_r=True)
)

# Generate
stl_file3 = ornament3.generate_and_save(
    filename='custom_scaling_ornament.stl',
    size_mm=75,
    verbose=True
)

print(f"✓ Created with custom scaling: {stl_file3}")

## Summary

You've learned how to:
1. Visualize complex functions on the Riemann sphere
2. Export them as STL files for 3D printing
3. Control the appearance with scaling methods
4. Handle difficult functions with domain restrictions
5. Create custom scaling for artistic effects

Now you can turn any complex function into a physical object!